<a href="https://colab.research.google.com/github/raghusamarth/SyntaxSquad_AIML/blob/main/jobgrind.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
#drive.mount('/content/drive')
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install transformers sentence-transformers
!pip install scrapy pandas transformers sentence-transformers scikit-learn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.2/254.2 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.9/104.9 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 6.3 MB/s eta 0:00:00


In [6]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [7]:
import pandas as pd

# Example dataset
data = {
    "job_id": [1, 2],
    "skills": ["Python; Java; SQL", "Python;TensorFlow;PyTorch"]
}
job_data = pd.DataFrame(data)
print(job_data.columns)
# Preprocess the 'skills' column
def preprocess_skills(skills):
    if pd.isnull(skills):  # Handle missing values
        return ""
    # Convert to lowercase, replace semicolons with commas, and strip extra spaces
    return skills.lower().replace(';', ', ').strip()

# Create the 'cleaned_skills' column
job_data['cleaned_skills'] = job_data['skills'].apply(preprocess_skills)

print(job_data)


Index(['job_id', 'skills'], dtype='object')
   job_id                     skills               cleaned_skills
0       1          Python; Java; SQL          python,  java,  sql
1       2  Python;TensorFlow;PyTorch  python, tensorflow, pytorch


In [8]:
import pandas as pd
import re

# Load the dataset
file_path = '/content/drive/MyDrive/JobGrind.io/job_grind/job_data.csv'  # Replace with the actual file path
job_data = pd.read_csv(file_path)
print(job_data.columns)
print(job_data.head())


def preprocess_text(text):
    if pd.isnull(text):  # Handle missing values
        return ""

    return re.sub(r'[^a-zA-Z\s]', '', text).lower()

job_data['cleaned_desription'] = job_data['description'].apply(preprocess_text)

def preprocess_skills(skills):
    if pd.isnull(skills):
        return ""
    return skills.lower().replace(';', ', ')

job_data['cleaned_skills'] = job_data['skills'].apply(preprocess_skills)

job_data['cleaned_location'] = job_data['location'].fillna('Unknown')

print(job_data[['cleaned_description', 'cleaned_skills', 'cleaned_location']].head())

cleaned_file_path = '/content/drive/MyDrive/JobGrind.io/job_grind/cleanjob_grind.csv'
job_data.to_csv(cleaned_file_path, index=False)
print(f"Cleaned dataset saved to {cleaned_file_path}")


Index(['job_id', 'title', 'description', 'skills', 'location', 'compensation',
       'platform', 'cleaned_description', 'cleaned_skills',
       'cleaned_location'],
      dtype='object')
   job_id                 title  \
0       1     Software Engineer   
1       2        Data Scientist   
2       3         Web Developer   
3       4  Full Stack Developer   
4       5        UI/UX Designer   

                                         description  \
0  Develop scalable software solutions using Pyth...   
1  Analyze datasets to build predictive models an...   
2  Create responsive websites with modern framewo...   
3  Develop both client-side and server-side appli...   
4  Design user interfaces for mobile and web appl...   

                      skills       location compensation   platform  \
0            Python;Java;SQL         Remote         Paid     Indeed   
1  Python;R;Machine Learning       New York         Paid   LinkedIn   
2  JavaScript;HTML;CSS;React  San Francisco       

In [9]:
# Update your code in the cell like this:
from sentence_transformers import SentenceTransformer

# Load the Sentence-BERT model
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Ensure all elements in the cleaned_description column are strings
job_data['cleaned_description'] = job_data['cleaned_description'].astype(str)

# Generate embeddings for job descriptions
job_embeddings = model.encode(job_data['cleaned_description'].tolist())

print(job_embeddings)

# Save embeddings to avoid recomputation
np.save('/content/drive/MyDrive/JobGrind.io/job_grind/job_embeddings.npy', job_embeddings)
print("Job embeddings generated and saved!")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

[[-0.4606768   0.52586603 -0.02412635 ...  0.26479402 -0.32959187
  -0.17524572]
 [-0.4606768   0.52586603 -0.02412635 ...  0.26479402 -0.32959187
  -0.17524572]
 [-0.4606768   0.52586603 -0.02412635 ...  0.26479402 -0.32959187
  -0.17524572]
 ...
 [-0.4606768   0.52586603 -0.02412635 ...  0.26479402 -0.32959187
  -0.17524572]
 [-0.4606768   0.52586603 -0.02412635 ...  0.26479402 -0.32959187
  -0.17524572]
 [-0.46067682  0.525866   -0.02412639 ...  0.26479396 -0.32959226
  -0.1752458 ]]
Job embeddings generated and saved!


In [10]:
from sentence_transformers import SentenceTransformer

# Load the Sentence-BERT model
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Example user input
user_skills = "Python, SQL, Machine Learning, Data Analysis"

# Preprocess user skills
def preprocess_text(text):
    import re
    return re.sub(r'[^a-zA-Z\s]', '', str(text)).lower()

user_skills_cleaned = preprocess_text(user_skills)

# Generate embedding for user skills
user_embedding = model.encode([user_skills_cleaned])

print("User embedding generated!")



User embedding generated!


In [11]:
#similarity check
import numpy as np

job_embeddings = np.load('/content/drive/MyDrive/JobGrind.io/job_grind/job_embeddings.npy')

print("Job embeddings loaded!")

similarity_scores = cosine_similarity(user_embedding, job_embeddings)

job_data['similarity_score'] = similarity_scores[0]
print(similarity_scores)
reccommended_jobs = job_data.sort_values(by='similarity_score', ascending=False)
print(reccommended_jobs.columns)
print(reccommended_jobs[['title', 'similarity_score']].head())


Job embeddings loaded!
[[0.11788923 0.11788923 0.11788923 0.11788923 0.11788923 0.11788923
  0.11788923 0.11788923 0.11788923 0.11788923 0.11788923 0.11788923
  0.11788921 0.11788921 0.11788927]]
Index(['job_id', 'title', 'description', 'skills', 'location', 'compensation',
       'platform', 'cleaned_description', 'cleaned_skills', 'cleaned_location',
       'cleaned_desription', 'similarity_score'],
      dtype='object')
                   title  similarity_score
14      Network Engineer          0.117889
0      Software Engineer          0.117889
1         Data Scientist          0.117889
2          Web Developer          0.117889
3   Full Stack Developer          0.117889


In [14]:
output_path = '/content/drive/MyDrive/JobGrind.io/job_grind/reccommended_jobs.csv'
reccommended_jobs[['title', 'similarity_score', 'description', 'location']].to_csv(output_path, index=False)
print(f"Recommended jobs saved to: {output_path}")

Recommended jobs saved to: /content/drive/MyDrive/JobGrind.io/job_grind/reccommended_jobs.csv
